In [102]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import time
from torch.nn import functional as F
from torch.nn import Sequential, Tanh, Parameter
import mlflow as mlflow
%matplotlib inline

In [2]:
#!curl https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt -o ../datasets/llm/shakeshpere.txt

In [3]:
shakspheredata= open("input.txt",mode="r",encoding="utf8").read()

In [4]:
vocab = sorted(list(set(shakspheredata)))
print("".join(vocab))
n_vocab = len(vocab)

stoi = {k:v for v,k in enumerate(vocab)}
itos = {v:k for v,k in enumerate(vocab)}
encode = lambda x:[stoi[i] for i in x]
decode = lambda x: "".join([itos[i] for i in x])


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [9]:
print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [54]:
#Data prep
text = torch.tensor(encode(shakspheredata))
n = int(.9*len(text))
train = text[:n]
val = text[n:]

In [18]:
block_size = 8
train[:block_size+1]

[18, 47, 56, 57, 58, 1, 15, 47, 58]

In [57]:
sample = train[:block_size+1]
for x in range(block_size):
    print(f"context {sample[:x+1]} : {sample[x+1]}")

context tensor([18]) : 47
context tensor([18, 47]) : 56
context tensor([18, 47, 56]) : 57
context tensor([18, 47, 56, 57]) : 58
context tensor([18, 47, 56, 57, 58]) : 1
context tensor([18, 47, 56, 57, 58,  1]) : 15
context tensor([18, 47, 56, 57, 58,  1, 15]) : 47
context tensor([18, 47, 56, 57, 58,  1, 15, 47]) : 58


In [99]:
batch_size = 4
block_size = 8

def get_batch(split):
    data = train if "train" else val
    ix =  torch.randint(len(data) - block_size ,(batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])

    return x,y

In [100]:
xbch,ybch = get_batch("train")

In [101]:
xbch,ybch

(tensor([[43, 51, 40, 43, 56,  1, 58, 46],
         [52, 45,  1, 40, 59, 58,  1, 39],
         [46, 43, 56, 43,  1, 47, 57,  1],
         [46, 39, 50, 44,  1, 57, 53,  1]]),
 tensor([[51, 40, 43, 56,  1, 58, 46, 43],
         [45,  1, 40, 59, 58,  1, 39,  1],
         [43, 56, 43,  1, 47, 57,  1, 52],
         [39, 50, 44,  1, 57, 53,  1, 45]]))

In [105]:
xbch.shape,ybch.shape

(torch.Size([4, 8]), torch.Size([4, 8]))

In [178]:
class BigramModel(nn.Module):
    def __init__(self,vocab):
        super().__init__()
        self.embedding_table = nn.Embedding(vocab,vocab)

    def forward(self,idx,target=None):
        logits = self.embedding_table(idx)
        if target == None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T,C)
            loss = F.cross_entropy(logits,target.view(B*T))
        return logits,loss

    def generate(self,max_tokens,idx):
        for _ in range(max_tokens):
            logits,loss = self(idx)
            logits = logits[:,-1,:]
            probs = F.softmax(logits,dim=-1)
            idx_next = torch.multinomial(probs,1)
            idx =torch.cat((idx,idx_next), dim=1)

        return idx

In [179]:
model = BigramModel(n_vocab)

In [180]:
logits,loss = model(xbch,ybch)

In [184]:
len(decode(model.generate(100,idx =torch.zeros((1, 1), dtype=torch.long))[0].tolist()))

101

In [185]:
F.softmax(logits[:,-1,:],dim=1).sum(dim=1)

IndexError: too many indices for tensor of dimension 2